# Linear regression

---

## Linear and Logistic Regressions: prediction and metrics


Applied Mathematical Modeling in Banking

---

---

## References

1. [SQLite in R. Datacamp](https://www.datacamp.com/community/tutorials/sqlite-in-r)
